In [10]:
import requests
import xml.etree.ElementTree as ET
import time

def data_collection(start_date, last_date, subject):
    # Specify the base URL of the OAI-PMH interface
    base_url = 'http://export.arxiv.org/oai2'

    # Specify the verb
    verb = 'ListRecords'

    # Specify the metadata format
    metadata_prefix = 'oai_dc'

    # Formulate the URL
    url = 'http://export.arxiv.org/oai2?verb=ListRecords&set=math&metadataPrefix=oai_dc&from=2009-01-01&until=2009-12-31'

    # An empty list to hold all the XML responses
    responses = []

    while url is not None:
        # Send the GET request
        response = requests.get(url)

        # If the request was successful, parse the data
        if response.status_code == 200:
            responses.append(response.text)
            root = ET.fromstring(response.text)
            token_element = root.find('.//{http://www.openarchives.org/OAI/2.0/}resumptionToken')
            if token_element is not None:
                token = token_element.text
                # Form the new url using the resumption token
                url = f'{base_url}?verb={verb}&resumptionToken={token}'
                print(token)
            else:
                url = None  # No more pages
        else:
            print(f'Request failed with status code {response.status_code}')
            url = None  # Exit the loop
        time.sleep(5)
    return responses

In [11]:
get_data('2009-01-01', '2009-12-31', 'math')

Request failed with status code 503


[]

In [2]:
import pickle
from datetime import datetime, timedelta

def save_to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

# Generate range of years
start_year = 2009
end_year = datetime.now().year  # Current year
years = list(range(start_year, end_year + 1))

# Loop over years
for year in years:
    start_date = f'{year}-01-01'
    if year == end_year:
        end_date = f'{year}-07-31'
    else:
        end_date = f'{year}-12-31'
    
    # Get data
    responses = get_data(start_date, end_date, 'math')
    
    # Save responses to pickle
    filename = f'responses_{year}.pkl'
    save_to_pickle(responses, filename)

    # Avoid rate limit
    time.sleep(5)  # Adjust this delay as necessary
